
<a href="https://colab.research.google.com/drive/1fgL7Ab7n3HC6roZWqrdvkX64nN6Lcm2Q?usp=sharing" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

 we performed continued fine-tuning of the TinyLlama 1.1B Chat model to create a specialized Mental Health Development Chatbot.
A synthetic dataset consisting of 500 carefully designed mental health support dialogues was generated, covering a range of topics including managing anxiety, stress, loneliness, self-esteem, emotional resilience, and mindfulness practices.
We formatted the dataset using the ChatML instruction-response style and fine-tuned the model for 100 steps using Unsloth’s optimized SFTTrainer.
After training, the fine-tuned TinyLlama chatbot was tested on new unseen prompts, successfully providing empathetic and relevant mental health advice.
The final chatbot demonstrated an ability to generate supportive, informative, and context-appropriate responses, showing that even a lightweight model like TinyLlama can be effectively adapted to specialized domains with efficient continued fine-tuning.

In [2]:
#1 Install libraries
!pip install -q unsloth "torch>=2.1" transformers peft datasets trl accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# Step 2: Import modules

from unsloth import FastLanguageModel
from datasets import Dataset
from transformers import TrainingArguments, pipeline
from trl import SFTTrainer
import torch
import pandas as pd
import random



In [5]:
# Step 3: Load TinyLlama-1.1B-Chat model

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length=2048,
    dtype=torch.float16,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
)


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.4.1 patched 22 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [6]:
# Step 4: Generate 500 new mental health samples

questions = [
    "How can I deal with anxiety before public speaking?",
    "What should I do if I feel isolated from my friends?",
    "How to handle panic attacks in crowded places?",
    "What are healthy ways to manage work stress?",
    "How can mindfulness improve my mental health?",
    "What should I do when I feel emotionally exhausted?",
    "How to support a colleague showing signs of burnout?",
    "How does physical exercise impact mental wellbeing?",
    "Tips for building a positive daily routine?",
    "How can I gently encourage someone to seek therapy?"
]

answers = [
    "Practice breathing techniques, visualize success, and take things one step at a time.",
    "Reach out to trusted people, join group activities, or volunteer for causes you enjoy.",
    "Focus on grounding yourself using techniques like naming five things you see or feel.",
    "Prioritize tasks, delegate when possible, and take regular short breaks.",
    "Mindfulness improves focus, reduces stress, and increases emotional regulation.",
    "Take time to rest, set boundaries, and seek support from friends or a professional.",
    "Offer to listen without judgment and suggest they speak to HR or a mental health counselor.",
    "Exercise releases endorphins which boost mood, reduce anxiety, and improve sleep.",
    "Wake up early, maintain consistent sleep times, and schedule time for hobbies.",
    "Express concern kindly, share resources, and respect their pace and decision."
]

# Create 500 examples
data = []
for _ in range(500):
    q = random.choice(questions)
    a = random.choice(answers)
    data.append({"instruction": q, "input": "", "output": a})

# Convert to Huggingface Dataset
df = pd.DataFrame(data)
mental_health_dataset = Dataset.from_pandas(df)

# Preview
mental_health_dataset[0]


{'instruction': 'How can I deal with anxiety before public speaking?',
 'input': '',
 'output': 'Offer to listen without judgment and suggest they speak to HR or a mental health counselor.'}

In [7]:
# Step 5: Format prompt-response pairs

def format_example(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output = example["output"]
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:"
    return {"prompt": prompt, "response": output}

formatted_dataset = mental_health_dataset.map(format_example).remove_columns(["instruction", "input", "output"])

# Preview
formatted_dataset[0]


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

{'prompt': '### Instruction:\nHow can I deal with anxiety before public speaking?\n\n### Response:',
 'response': 'Offer to listen without judgment and suggest they speak to HR or a mental health counselor.'}

In [8]:
# Step 6: Fine-tune with SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="prompt",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        num_train_epochs=1,
        learning_rate=2e-5,
        fp16=True,
        max_steps=100,    # Only 100 steps
        logging_steps=10,
        output_dir="./tinyllama_mentalhealth_finetuned",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
)

trainer.train()


Unsloth: Tokenizing ["prompt"] (num_proc=2):   0%|          | 0/500 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 500 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 4,505,600/4,000,000,000 (0.11% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pomacoc217 (pomacoc217-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.161800
20,2.911600
30,2.727100
40,2.583500
50,2.416700
60,2.271600
70,2.174500
80,2.000200
90,1.913900
100,1.948900


TrainOutput(global_step=100, training_loss=2.411004981994629, metrics={'train_runtime': 98.5881, 'train_samples_per_second': 4.057, 'train_steps_per_second': 1.014, 'total_flos': 58987128987648.0, 'train_loss': 2.411004981994629})

In [9]:
# Step 7: Save fine-tuned mental health chatbot

model.save_pretrained("tinyllama_mentalhealth_chatbot")
tokenizer.save_pretrained("tinyllama_mentalhealth_chatbot")


('tinyllama_mentalhealth_chatbot/tokenizer_config.json',
 'tinyllama_mentalhealth_chatbot/special_tokens_map.json',
 'tinyllama_mentalhealth_chatbot/tokenizer.model',
 'tinyllama_mentalhealth_chatbot/added_tokens.json',
 'tinyllama_mentalhealth_chatbot/tokenizer.json')

In [10]:
# Step 8: Test the chatbot

def generate_response(prompt, max_tokens=128):
    chat_prompt = f"### Instruction:\n{prompt}\n\n### Response:"
    inputs = tokenizer(chat_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.strip()

# Example tests
print("\nTest 1:")
print(generate_response("How can mindfulness improve mental health?"))

print("\nTest 2:")
print(generate_response("What are healthy ways to deal with work stress?"))



Test 1:
### Instruction:
How can mindfulness improve mental health?

### Response:
Mindfulness has been shown to have a positive impact on mental health, including reducing symptoms of anxiety and depression. By focusing on the present moment and being fully present in the moment, individuals are better able to manage their thoughts and emotions, leading to improved well-being. Mindfulness also helps to reduce stress and promote relaxation, which can help alleviate symptoms of anxiety and depression. In addition, mindfulness practices such as meditation and yoga have been found to increase self-awareness, which can lead to greater self-compassion and em

Test 2:
### Instruction:
What are healthy ways to deal with work stress?

### Response:
1. Take breaks - take a break from work and do something that you enjoy, such as reading or taking a walk.
2. Practice mindfulness - try meditation or deep breathing exercises to help calm your mind and reduce stress.
3. Prioritize self-care - prio